# Bayesian analysis and MCMC

## Defining priors

In [4]:
from likelihood import JointPriors

# Defining the priors and the type of distribuition
param_configs_w_frb = {
    'H0': ((20, 100), 'uniform'),
    'Omega_bh2': ((0, 0.05), 'uniform'), 
    'Omega_m': ((0.1, 0.5), 'uniform'),
    'DM_host_0': ((55, 225), 'uniform'),
    'omega_0': ((-2, 0), 'uniform')
}

param_configs_CPL_frb = {
    'H0': ((20, 100), 'uniform'),
    'Omega_bh2': ((0, 0.05), 'uniform'), 
    'Omega_m': ((0.1, 0.5), 'uniform'),
    'DM_host_0': ((55, 225), 'uniform'),
    'omega_0': ((-2, 0), 'uniform'),
    'omega_a': ((-2, 2), 'uniform')
}

from desilike.theories import Cosmoprimo

# Configurando o modelo cosmológico
cosmo = Cosmoprimo(engine='camb')

## Set priors
cosmo.init.params = {'Omega_m': {'prior': {'limits': [0.1, 0.9]}},
                     
                     'H0': {'prior': {'limits': [20, 100]}},

                     'w0_fld': {'prior': {'limits': [-2, -0.3]}}
                     }

# Convertendo os parâmetros do cosmoprimo
cosmo_params_converted = JointPriors.convert_cosmo_params(cosmo.init.params)

"""param_configs_w_sne = {
    'H0': ((20, 100), 'uniform'),
    'Omega_m': ((0.1, 0.5), 'uniform'),
    'omega_0': ((-2, 0), 'uniform')
}"""

"""param_configs_CPL_sne = {
    'H0': ((20, 100), 'uniform'),
    'Omega_m': ((0.1, 0.5), 'uniform'),
    'omega_0': ((-2, 0), 'uniform'),
    'omega_a': ((-2, 2), 'uniform')
}"""

P_w_frb = JointPriors(param_configs_w_frb)
# Instanciando o JointPriors corretamente
P_w_sne = JointPriors(cosmo_params_converted)
#P_CPL_frb = JointPriors(param_configs_CPL_frb)
#P_CPL_sne = JointPriors(param_configs_CPL_sne)

print(cosmo_params_converted)

{'Omega_m': ((0.1, 0.9), 'uniform'), 'H0': ((20, 100), 'uniform'), 'w0_fld': ((-2, -0.3), 'uniform')}


### Models and parameters

In [2]:
from likelihood import JointLikelihoodFunction
from equations import DM_EXT_model, Modulus_sne
from desilike.likelihoods.supernovae import (
    PantheonPlusSHOESSNLikelihood, PantheonPlusSNLikelihood, PantheonSNLikelihood
)
from desilike import setup_logging

setup_logging()  # set up logging

# Configurando as likelihoods do desilike
desilike_likelihoods = {
    #'Pantheon': PantheonSNLikelihood(cosmo=cosmo),
    #'Pantheon+': PantheonPlusSNLikelihood(cosmo=cosmo),
    'Pantheon+ & SH0ES': PantheonPlusSHOESSNLikelihood(cosmo=cosmo)
}


model = DM_EXT_model()
#mu_wCDM = Modulus_sne()

# Creating an instance of JointLikelihoodFunction
LF_w_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2,  
        Omega_m=Omega_m,
        H0=H0,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

"""LF_w_sne = JointLikelihoodFunction(
    { 'SNe': lambda z, H0, Omega_m: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        #omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)"""

LF_w_sne = JointLikelihoodFunction(
    
    desilike_likelihoods=desilike_likelihoods 
    
)

"""LF_w_sne_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        H0=H0, 
        Omega_m=Omega_m,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ),

    'SNe': lambda z, H0, Omega_m, omega_0: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    },

    shared_params= ["H0", "Omega_m", "omega_0"]
)"""

LF_w_sne_frb = JointLikelihoodFunction(
    model_funcs = { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        H0=H0, 
        Omega_m=Omega_m,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ) },

    desilike_likelihoods=desilike_likelihoods,

    shared_params= ["H0", "Omega_m", "omega_0"]
)

"""LF_CPL_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        H0=H0, 
        Omega_m=Omega_m,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_CPL_sne = JointLikelihoodFunction(
    { 'SNe': lambda z, H0, Omega_m, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_CPL_sne_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2,  
        H0=H0,
        Omega_m=Omega_m,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='CPL'
    ),

    'SNe': lambda z, H0, Omega_m, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    },
    shared_params= ["H0", "Omega_m", "omega_0", "omega_a"]
)"""

'LF_CPL_frb = JointLikelihoodFunction(\n    { \'FRB\': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0, omega_a: model.DM_ext_th(\n        z=z,\n        f_IGM=0.83,\n        model_type=\'constant\',\n        Omega_bh2=Omega_bh2, \n        H0=H0, \n        Omega_m=Omega_m,     \n        DM_host_0=DM_host_0,\n        omega_0=omega_0,\n        omega_a=omega_a,  \n        cosmo_type=\'non_standard\',\n        param_type=\'CPL\'\n    ) \n    }\n)\n\nLF_CPL_sne = JointLikelihoodFunction(\n    { \'SNe\': lambda z, H0, Omega_m, omega_0, omega_a: mu_wCDM.Modulo_std(\n        z=z,\n        Omega_m=Omega_m,\n        H_0=H0,\n        omega_0=omega_0,\n        omega_a=omega_a,\n        cosmo_type=\'non_standard\',\n        param_type=\'CPL\'\n    ) \n    }\n)\n\nLF_CPL_sne_frb = JointLikelihoodFunction(\n    { \'FRB\': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0, omega_a: model.DM_ext_th(\n        z=z,\n        f_IGM=0.83,\n        model_type=\'constant\',\n        Omega_bh2=Omega_bh2

## Preparing the samples

### Analysis for 66 FRBs

In [3]:
from obs_data import FRB_data, SNe_data
import ultranest

# Instantiate the FRB_data class for 66 FRBs
frb_data = FRB_data(n_frb=66)

"""sne_data = SNe_data(sample_sne='Pantheon+')

z_sne, mu_sne, cov_matrix = sne_data.load_data()"""

# Call the select_data method to get the observed data
z_frb, DM_ext, sigma_ext = frb_data.select_data()

# Configuring the ultranest samplers
sampler_w_frb = ultranest.ReactiveNestedSampler(
    P_w_frb.param_names,
    lambda params: LF_w_frb.log_likelihood(
        dict(zip(P_w_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_w_frb.prior_transform
)

"""sampler_w_sne = ultranest.ReactiveNestedSampler(
    P_w_sne.param_names,
    lambda params: LF_w_sne.log_likelihood(
        dict(zip(P_w_sne.param_names, params)),
        {
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w_sne.prior_transform
)"""

sampler_w_sne = ultranest.ReactiveNestedSampler(
    P_w_sne.param_names,
    lambda params: LF_w_sne.log_likelihood(
        dict(zip(P_w_sne.param_names, params))
    ),
    P_w_sne.prior_transform
)

"""sampler_w_sne_frb = ultranest.ReactiveNestedSampler(
    P_w_frb.param_names,
    lambda params: LF_w_sne_frb.log_likelihood(
        dict(zip(P_w_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext),
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w_frb.prior_transform
)"""

sampler_w_sne_frb = ultranest.ReactiveNestedSampler(
    P_w_frb.param_names,
    lambda params: LF_w_sne_frb.log_likelihood(
        dict(zip(P_w_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_w_frb.prior_transform
)

"""sampler_CPL_frb = ultranest.ReactiveNestedSampler(
    P_CPL_frb.param_names,
    lambda params: LF_CPL_frb.log_likelihood(
        dict(zip(P_CPL_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_CPL_frb.prior_transform
)

sampler_CPL_sne = ultranest.ReactiveNestedSampler(
    P_CPL_sne.param_names,
    lambda params: LF_CPL_sne.log_likelihood(
        dict(zip(P_CPL_sne.param_names, params)),
        {
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_CPL_sne.prior_transform
)

sampler_CPL_sne_frb = ultranest.ReactiveNestedSampler(
    P_CPL_frb.param_names,
    lambda params: LF_CPL_sne_frb.log_likelihood(
        dict(zip(P_CPL_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext),
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_CPL_frb.prior_transform
)"""

[000000.23] [0/1] 10-24 00:24  ultranest                 DEBUG    ReactiveNestedSampler: dims=5+0, resume=False, log_dir=None, backend=hdf5, vectorized=False, nbootstraps=30, ndraw=128..65536
[000000.24] [0/1] 10-24 00:24  ultranest                 DEBUG    ReactiveNestedSampler: dims=3+0, resume=False, log_dir=None, backend=hdf5, vectorized=False, nbootstraps=30, ndraw=128..65536
[000000.24] [0/1] 10-24 00:24  PantheonPlusSHOESSNLikelihood INFO     Loading light-curve from /home/lazarolima/.local/data/PantheonPlusSNLikelihood/Pantheon+SH0ES.dat
[000000.28] [0/1] 10-24 00:24  PantheonPlusSHOESSNLikelihood INFO     Loading covariance from /home/lazarolima/.local/data/PantheonPlusSNLikelihood/Pantheon+SH0ES_STAT+SYS.cov
Erro ao calcular likelihood Pantheon+ & SH0ES: calculator PantheonPlusSHOESSNLikelihood has no attribute loglikelihood; have you run any calculation already by calling this calculator or calculators that depend on it (typically, a likelihood?)
Erro ao calcular likelihood 

AssertionError: Error in loglikelihood function: returned non-finite number: [-inf -inf] for input u=[[0.35434477 0.23140245 0.29421023]
 [0.98674544 0.82141922 0.75774016]] p=[[ 0.38347582 38.51219605 -1.49984261]
 [ 0.88939636 85.7135374  -0.71184173]]

In [ ]:
wCDM_frb = sampler_w_frb.run(min_num_live_points=400)
sampler_w_frb.print_results()

In [ ]:
wCDM_sne = sampler_w_sne.run(min_num_live_points=400)
sampler_w_sne.print_results()

In [ ]:
wCDM_sne_frb = sampler_w_sne_frb.run(min_num_live_points=400)
sampler_w_sne_frb.print_results()

In [ ]:
CPL_frb = sampler_CPL_frb.run(min_num_live_points=400)
sampler_CPL_frb.print_results()

In [ ]:
CPL_sne = sampler_CPL_sne.run(min_num_live_points=400)
sampler_CPL_sne.print_results()

In [ ]:
CPL_sne_frb = sampler_CPL_sne_frb.run(min_num_live_points=400)
sampler_CPL_sne_frb.print_results()

In [ ]:
from getdist import plots, MCSamples

# Extraindo amostras dos resultados para frb
samples1_frb = wCDM_frb['samples']
samples2_frb = CPL_frb['samples']

labels1 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0']
names1 = P_w_frb.param_names
mcsamples1_frb = MCSamples(samples=samples1_frb, names=names1, labels=labels1)

labels2 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0', '\\omega_a']
names2 = P_CPL_frb.param_names
mcsamples2_frb = MCSamples(samples=samples2_frb, names=names2, labels=labels2)

# Extraindo amostras dos resultados para sne
samples1_sne = wCDM_sne['samples']
samples2_sne = CPL_sne['samples']

labels3 = ['H_0', '\\Omega_m', '\\omega_0']
names3 = P_w_sne.param_names
mcsamples1_sne = MCSamples(samples=samples1_sne, names=names3, labels=labels3)

labels4 = ['H_0', '\\Omega_m', '\\omega_0', '\\omega_a']
names4 = P_CPL_sne.param_names
mcsamples2_sne = MCSamples(samples=samples2_sne, names=names4, labels=labels4)

# Extraindo amostras dos resultados para sne+frb

samples1_sne_frb = wCDM_sne_frb['samples']
samples2_sne_frb = CPL_sne_frb['samples']

mcsamples1_sne_frb = MCSamples(samples=samples1_sne_frb, names=names1, labels=labels1)
mcsamples2_sne_frb = MCSamples(samples=samples2_sne_frb, names=names2, labels=labels2)



In [30]:
import pandas as pd

# Dicionário de amostras e nomes de arquivos
amostras = {
    "samples1_frb": samples1_frb,
    "samples2_frb": samples2_frb,
    "samples1_sne": samples1_sne,
    "samples2_sne": samples2_sne,
    "samples1_sne_frb": samples1_sne_frb,
    "samples2_sne_frb": samples2_sne_frb,
}

# Salva cada conjunto de amostras em arquivos CSV
for nome, dados in amostras.items():
    df = pd.DataFrame(dados)
    df.to_csv(f"Samplers/{nome}.csv", index=False)


In [7]:
import pandas as pd
df1_frb = pd.read_csv("Samplers/samples1_frb.csv")
df1_sne = pd.read_csv("Samplers/samples1_sne.csv")
df1_sne_frb = pd.read_csv("Samplers/samples1_sne_frb.csv")
df2_frb = pd.read_csv("Samplers/samples2_frb.csv")
df2_sne = pd.read_csv("Samplers/samples2_sne.csv")
df2_sne_frb = pd.read_csv("Samplers/samples2_sne_frb.csv")

In [ ]:
from getdist import plots, MCSamples

# Extraindo amostras dos resultados para frb
samples1_frb = df1_frb.values
samples2_frb = df2_frb.values

labels1 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0']
names1 = P_w_frb.param_names
mcsamples1_frb = MCSamples(samples=samples1_frb, names=names1, labels=labels1)

labels2 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0', '\\omega_a']
names2 = P_CPL_frb.param_names
mcsamples2_frb = MCSamples(samples=samples2_frb, names=names2, labels=labels2)

# Extraindo amostras dos resultados para sne
samples1_sne = df1_sne.values
samples2_sne = df2_sne.values

labels3 = ['H_0', '\\Omega_m', '\\omega_0']
names3 = P_w_sne.param_names
mcsamples1_sne = MCSamples(samples=samples1_sne, names=names3, labels=labels3)

labels4 = ['H_0', '\\Omega_m', '\\omega_0', '\\omega_a']
names4 = P_CPL_sne.param_names
mcsamples2_sne = MCSamples(samples=samples2_sne, names=names4, labels=labels4)

# Extraindo amostras dos resultados para sne+frb

samples1_sne_frb = df1_sne_frb.values
samples2_sne_frb = df2_sne_frb.values

mcsamples1_sne_frb = MCSamples(samples=samples1_sne_frb, names=names1, labels=labels1)
mcsamples2_sne_frb = MCSamples(samples=samples2_sne_frb, names=names2, labels=labels2)



In [15]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples1_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples2_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples1_frb, mcsamples2_frb], filled=True, contour_colors=['green', 'red'], 
                legend_labels=['$\omega_0$CDM with FRB', '$\omega_0\omega_a$CDM with FRB'])
g.export('Figuras/wCDM_CPL_frb.png', dpi=600)

In [21]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
#mcsamples1_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples1_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples1_sne_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples1_sne_frb, mcsamples1_sne], 
                params=['H0', 'Omega_m', 'omega_0'] , filled=True, 
                contour_colors=['red', 'green'], 
                legend_labels=['$\omega_0$CDM with SNe + FRB', '$\omega_0$CDM with SNe'])
g.export('Figuras/param_constant_cosmo.png', dpi=600)

In [22]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples2_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples2_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples2_sne_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples2_sne_frb, mcsamples2_sne], 
                params=['H0', 'Omega_m', 'omega_0', 'omega_a'] , filled=True, 
                contour_colors=['red', 'green'], 
                legend_labels=['$\omega_0\omega_a$CDM with SNe + FRB', '$\omega_0\omega_a$CDM with SNe'])
g.export('Figuras/param_CPL_cosmo.png', dpi=600)

## Model Comparison

In [22]:
samples = {
    "Sample_frb": {
        "results": {
            'wCDM': {'logz': wCDM_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_frb['logz'], 'num_params': 6},
        }
    },
    "Sample_sne": {
        "results": {
            'wCDM': {'logz': wCDM_sne['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne['logz'], 'num_params': 6},
        }
    },
    "Sample_sne_frb": {
        "results": {
            'wCDM': {'logz': wCDM_sne_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne_frb['logz'], 'num_params': 6},
        }
    }
}

In [27]:
from bayesian_analysis import ModelComparison

comparison = ModelComparison(samples)
comparison.run_comparisons(save_to_file='comparisons_output_frb+SNe.txt')

In [28]:
from bayesian_analysis import SaveResults

# Initialize the class with the desired output file
results_saver = SaveResults("model_results.txt")

# Optional: Clear the file if you want to start fresh
results_saver.reset_file()

results_saver.save_to_txt(mcsamples1_frb, 'wCDM_frb')
results_saver.save_to_txt(mcsamples2_frb, 'CPL_frb')
results_saver.save_to_txt(mcsamples1_sne, 'wCDM_sne')
results_saver.save_to_txt(mcsamples2_sne, 'CPL_sne')
results_saver.save_to_txt(mcsamples1_sne_frb, 'wCDM_sne_frb')
results_saver.save_to_txt(mcsamples2_sne_frb, 'CPL_sne_frb')


In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_16, y=dm_ext_obs_16, fmt='o', alpha=0.6, color='red', label='16 FRBs', ms=2)

z_test = np.linspace(0, 1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0, omega_a  in sampler_p2_16.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,  
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'))

band.line(color='k', linestyle='-', label='CPL parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_16.png', format='png', dpi=600)"""

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_63, y=dm_ext_obs_63, fmt='o', alpha=0.6, color='red', label='63 FRBs', ms=2)

z_test = np.linspace(0, 1.1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0  in sampler_constant_63.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant')
    )

band.line(color='k', linestyle='-', label='Constant parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_63.png', format='png', dpi=600)"""